# About Ray

<img src='images/ray_header_logo.png' width=600/>

### Introduction

#### What is Ray?

<div class="alert alert-info">
  <strong><a href="https://www.ray.io/" target="_blank">Ray</a></strong> is an open-source unified compute framework that makes it easy to scale AI and Python workloads.
</div>

Thanks to the Python first approach, ML engineers can parallelize Python applications on their laptop, cluster, cloud, Kubernetes, or on-premise hardware. Ray automatically handles all aspects of distributed execution including orchestration, scheduling, fault tolerance, and auto-scaling so that you can scale your apps without becoming a distributed systems expert.

With a rich ecosystem of libraries and integrations with many important data science tools, Ray lowers the effort needed to scale compute intensive workloads and applications.

#### Distributed computing: a bit of a context and project history

|<img src="images/project_history.jpeg" width="70%" loading="lazy">|
|:--|
|Compute demand is growing faster than supply. It exceeds progression of CPUs, GPUs and TPUs processing power. (date accessed: Nov 2, 2022)|

Distributed computing is hard. At the same time it is becoming increasingly crucial and necessary for modern machine learning and AI systems.

OpenAI's recent paper [AI and Compute](https://openai.com/blog/ai-and-compute/) suggests exponential growth in compute needed to train AI models. Study suggests that compute needed for AI systems has been doubling every 3.4 months since 2012.

This context drove researchers to begin building solutions to simplify running code on compute clusters without having to think about how to orchestrate and utilize individual machines. That is, let Ray do the hard bit of orchestrating and executing, and you do the easy bit of writing Python code.

Ray was developed at the University of California Berkeley's [RISELab](https://rise.cs.berkeley.edu/), the successor to the [AMPLab](https://amplab.cs.berkeley.edu/about/), that created [Apache Spark](https://spark.apache.org/) and [Mesos](https://mesos.apache.org/). 

[Anyscale](https://www.anyscale.com/), the company behind Ray, was founded by Ray creators to build a managed Ray platform and offers hosted solutions for Ray applications.

### Key Ray characteristics

|<img src="images/python_first.jpeg" width="70%" loading="lazy">|<img src="images/simple_and_flexible_api.jpeg" width="70%" loading="lazy">|<img src="images/scalability.jpeg" width="70%" loading="lazy">|<img src="images/heterogeneous_hardware.jpeg" width="70%" loading="lazy">|
|:-:|:-:|:-:|:-:|
|Python first approach|Simple and Flexible API|Scalability|Support for heterogeneous hardware|

#### Python first approach

<img src="images/python_first.jpeg" width="100px" loading="lazy">

Ray's framework provides Python library with abstractions and primitives that enables ML practitioners and Python developers to build distributed applications. Ray exposes concise and easy to use API. Its core library that enables parallel execution introduces just a few key abstractions:

1. [Tasks](https://docs.ray.io/en/latest/ray-core/key-concepts.html#tasks): remote, stateless Python functions
1. [Actors](https://docs.ray.io/en/latest/ray-core/key-concepts.html#actors): remote stateful Python classes
1. [Objects](https://docs.ray.io/en/latest/ray-core/key-concepts.html#objects): in-memory, immutable objects or value that can be accessed anywhere in the computing cluster

#### Simple and flexible API

<img src="images/simple_and_flexible_api.jpeg" width="100px" loading="lazy">

##### Ray Core

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/walkthrough.html" target="_blank">Ray Core</a></strong> is an open-source, Python, general purpose, distributed computing library that enables ML engineers and Python developers to scale Python applications and accelerate machine learning workloads.
</div>

Foundational library for the whole ecosystem - provides minimalist API that enables distributed computing. With just a few methods you can start building distributed apps.

* `ray.init()` - start Ray runtime and connect to the Ray cluster
* `@ray.remote` -  functions and classes decorator specifying that it will be executed as a task (remote function) or actor (remote class) in a different process
* `.remote` - postfix to the remote functions and classes. Remote operations are *asynchronous*
* `ray.put()` - put an object in the in-memory object store and return its ID. Use this ID to pass object to any remote function or method call
* `ray.get()` - get a remote object or a list of remote objects from the object store

##### Ray AI Runtime (AIR)

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-air/getting-started.html" target="_blank">Ray AI Runtime (AIR)</a></strong> is an open-source, Python, domain specific library that equips ML engineers, data scientists, and researchers with a scalable and unified toolkit for ML applications.
</div>

Ray AI Runtime (AIR) (sometimes referred to as native libraries) and ecosystem libraries, provides higher level APIs that cater for more domain specific use cases. Ray AIR enables Python developers and ML engineers to scale individual workloads, end-to-end workflows, and popular ecosystem frameworks, all in familiar Python programming language.

#### Scalability

<img src="images/scalability.jpeg" width="100px" loading="lazy">

Ray allows their users to utilize large compute clusters in an easy, productive, and resource-efficient way.

Fundamentally, Ray treats the entire cluster as a single, unified pool of resources and takes care of optimally mapping compute workloads to the pool. By doing so, Ray largely eliminates non-scalable factors in the system. Successful user stories include, but are not limited to:
* [how Instacart uses Ray to power their large scale fulfillment ML pipline](https://www.youtube.com/watch?v=3t26ucTy0Rs),
* [how OpenAI trains their largest models](https://twitter.com/anyscalecompute/status/1562136159135973380),
* [how companies like HuggingFace and Cohere use Ray Train for scaling model training](https://www.youtube.com/watch?v=For8yLkZP5w).

Ray's [autoscaler](https://docs.ray.io/en/latest/cluster/key-concepts.html#autoscaling) implements automatic scaling of Ray clusters based on the resource demands of an application. The autoscaler will increase worker nodes when the Ray workload exceeds the cluster's capacity. Whenever worker nodes sit idle, the autoscaler will scale them down.

#### Support for heterogeneous hardware

<img src="images/heterogeneous_hardware.jpeg" width="100px" loading="lazy">

One of the key properties of Ray is natively supporting heterogeneous hardware by allowing developers to specify such hardware when instantiating a Task or Actor. For example, a developer can specify in the same application that one Task needs 128 CPUs, while an Actor requires 36 CPUs and 8 Nvidia A100 GPUs.

An illustrative example is the [production deep learning pipeline at Uber](https://www.anyscale.com/ray-summit-2022/agenda/sessions/215). A heterogeneous setup of 8 GPU nodes and 9 CPU nodes improves the pipeline throughput by 50%, while substantially saving capital cost, compared with the legacy setup of 16 GPU nodes.

|<img src="images/uber.png" width="70%" loading="lazy">|
|:--|
|Production deep learning pipeline at Uber.|

Let's jump in and take a very quick look at code to
* access data
* train a (distributed) XGBoost model
* serve that model

In [ ]:
import ray

ray.init()

In [ ]:
dataset = ray.data.read_parquet("s3://anyscale-training-data/intro-to-ray-air/nyc_taxi_2021.parquet")

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

train_dataset = train_dataset.repartition(num_blocks=5)
valid_dataset = valid_dataset.repartition(num_blocks=5)

In [ ]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    label_column="is_big_tip",
    num_boost_round=20,
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
    params={
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "tree_method": "approx",
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

In [ ]:
result = trainer.fit()

In [ ]:
result

In [ ]:
from ray import serve
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json
from ray.train.xgboost import XGBoostPredictor

serve.run(
    PredictorDeployment.options(
        name="XGBoostService", num_replicas=2, route_prefix="/rayair"
    ).bind(XGBoostPredictor, result.checkpoint, http_adapter=pandas_read_json)
)

In [ ]:
import requests

sample_input = train_dataset.take(1)

sample_input

In [ ]:
sample_input = dict(sample_input[0])
del(sample_input['is_big_tip'])
del(sample_input['__index_level_0__'])

output = requests.post("http://localhost:8000/rayair", json=[sample_input]).json()
print(output)

In [ ]:
ray.shutdown()

|<img src="images/map.png" width="70%" loading="lazy">|
|:--|
|Stack of Ray libraries - unified toolkit for ML workloads.|

#### Ray AI Runtime

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-air/getting-started.html" target="_blank">Ray AI Runtime (AIR)</a></strong> is an open-source, Python, domain specific library that equips ML engineers, data scientists, and researchers with a scalable and unified toolkit for ML applications.
</div>

Ray AIR is built on top of Ray core. It caters for distributed data processing, model training, tuning, model serving, and reinforcement learning, all in Python. To that end, it enables both individual workloads and end-to-end use cases to be implemented in the single unified library.

Ray AIR brings together an ever-growing ecosystem of integrations with your favorite machine learning frameworks.

|<img src="images/e2e_air.png" width="70%" loading="lazy">|
|:--|
|Ray AIR enables end-to-end ML development and provides multiple options to integrate with other tools and libraries form the MLOps ecosystem.|

Each of the five native libraries that Ray AIR wraps is focused on a specific ML task. Because this abstraction layer is built on top of Ray Core, it is distributed and scalable.

1. [Ray Data](https://docs.ray.io/en/latest/data/dataset.html): scalable, framework-agnostic loading and transforming raw data across training and prediction
1. [Ray Train](https://docs.ray.io/en/latest/train/train.html): distributed multi-node and multi-core model training with fault tolerance that integrates with your favorite training libraries
1. [Ray Tune](https://docs.ray.io/en/latest/tune/index.html): scales experiment execution and hyperparameter tuning to optimize model performance
1. [Ray Serve](https://docs.ray.io/en/latest/serve/index.html): deploys your model for online inference, with optional microbatching to improve performance
1. [Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html): distributed reinforcement learning workloads that integrate with the other Ray AIR libraries above

#### Integrations and ecosystem libraries

Ray integrates with a [growing ecosystem](https://docs.ray.io/en/latest/ray-overview/ray-libraries.html) of the most popular Python and machine learning libraries and frameworks that you may already be familiar with. Instead of trying to create new standards, Ray allows you to scale existing workloads by unifying tools in a common interface. This interface enables you to run ML tasks in a distributed way, a property most of the respective backends don't have, or not to the same extent.

For example, Ray Datasets is backed by Arrow and comes with many integrations to other frameworks, such as Spark and Dask. Ray Train and RLlib are backed by the full power of Tensorflow and PyTorch. Ray Tune supports algorithms from practically every noteable HPO tool available, including Hyperopt, Optuna, Nevergrad, Ax, SigOpt, and many others. Ray Serve can be used with frameworks such as FastAPI, gradio, and Streamlit.